<a href="https://colab.research.google.com/github/Witcape/Emotion_Recognition_in_Live_Video_Data/blob/main/Emotion_Recognition_in_Video_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### STEP 1 - Import all the Libraries and google drive

In [21]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import sklearn
import tensorflow as tf
import pandas as pd
import sklearn
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.activations import linear,relu,sigmoid
import cv2
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: ignored

## Step 2 - Loading Data & Preprocessing

In [3]:
train_data_dir = '/content/gdrive/MyDrive/Human emotions/train'
test_data_dir = '/content/gdrive/MyDrive/Human emotions/test'

In [4]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir)
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir)

Found 29031 files belonging to 7 classes.
Found 7378 files belonging to 7 classes.


In [5]:
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

In [6]:
train_batch = train_data_iterator.next()
test_batch = test_data_iterator.next()

In [7]:
test_batch

(array([[[[ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          ...,
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.]],
 
         [[ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          ...,
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.]],
 
         [[ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          [ 21.,  21.,  21.],
          ...,
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.],
          [ 53.,  53.,  53.]],
 
         ...,
 
         [[ 83.,  83.,  83.],
          [ 83.,  83.,  83.],
          [ 83.,  83.,  83.],
          ...,
          [ 97.,  97.,  97.],
          [ 97.,  97.,  97.],
          [ 97.,  97.,  97.]],
 
         [[ 83.,  83.,  83.],
          [ 83.,  83.,  83.],
          [ 83.,  83.,  83.],
          ...,
          [ 97.,  97.,  97.],
          [ 97.,  97.,  97.],
          [ 97.,  97.,  97.

In [8]:
len(test_batch)

2

In [9]:
train_batch[0].shape

(32, 256, 256, 3)

In [10]:
train_data = train_data.map(lambda x,y: (x/255,y))
test_data = test_data.map(lambda x,y: (x/255,y))

In [11]:

scaled_train_iterator = train_data.as_numpy_iterator()
scaled_test_iterator = test_data.as_numpy_iterator()
train_batch = scaled_train_iterator.next()
test_batch = scaled_test_iterator.next()
test_batch[0].max()

1.0

In [12]:
len(train_data)

908

In [13]:
train_size = int(len(train_data)*0.7)
val_size = int(len(train_data)*0.3)+1
test_size = len(test_data)
train_size+val_size

908

In [30]:
train = train_data.take(train_size)
val = train_data.skip(train_size).take(val_size)
test = test_data.take(test_size)

In [15]:
print(len(train),len(val),len(test))

635 273 231


## Step 3 - Model

In [23]:
model = Sequential(
    [
        Conv2D(32, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(64, (3,3), 1,  activation = 'relu'),
        MaxPooling2D(),
        Dropout(0.25),

        Conv2D(16, (3,3), 1,  activation = 'relu', input_shape = (256, 256, 3)),
        MaxPooling2D(),
        Dropout(0.25),

        Flatten(),

        Dense(256, activation = 'relu'),
        Dense(7, activation = 'softmax')
    ]
)

In [24]:
model.compile(
    optimizer = Adam(learning_rate = 0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [25]:
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3, monitor='val_loss', min_lr=0.00001)

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 62, 62, 64)        0

In [27]:
log_dir = '/content/gdrive/MyDrive/Human emotions/Logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)

In [1]:
history = model.fit(train, validation_data = val, epochs = 20, callbacks = [tensorboard_callback,  reduce_lr])

NameError: ignored

In [ ]:
model.save('/content/gdrive/MyDrive/Human emotions/Saved_model')


In [28]:
loaded_model = load_model('/content/gdrive/MyDrive/Human emotions/Saved_model')


In [31]:
predictions = loaded_model.predict(test)

231/231 [==============================] - 977s 4s/step


In [34]:
predicted_classes = np.argmax(predictions, axis=1)

In [33]:
test_labels = np.concatenate([y for x, y in test], axis=0)

In [35]:
correct_predictions = np.equal(predicted_classes, test_labels)

In [36]:
accuracy = np.mean(correct_predictions)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.17647058823529413
